In [1]:
import requests
from keys import API_KEY, APP_KEY

In [5]:
def request_url(*param, req='search', form='json', api_url='http://api.duma.gov.ru/api/', 
                token=API_KEY, app_token=APP_KEY, mode=None):
    body = api_url + token + '/' + req + '.' + form + '?' + 'app_token=' + app_token
    for i in param:
        body = body + '&' + i
    
    try:
        response = requests.get(body)
        if response.status_code == 200:
            print('200 Success!')
        elif response.status_code == 404:
            print('404 Not Found!')
        response.raise_for_status()
    except HTTPError as http_err:
        print('HTTP error occurred: {}'.format(http_err))
    except Exception as err:
        print('Other error occurred: {}'.format(err))

    if mode == 'print':
        for j in response.json().items():
            print(str(j[0]) + ': ' + str(j[1]))
        
    return response

In [6]:
response = request_url('status=7')

200 Success!


In [7]:
response.text

'{"count":8235,"page":1,"wording":"Законопроекты, подписанные Президентом РФ, отсортированные по дате последнего события (по убыванию)","laws":[{"id":31930,"number":"904447-7","name":"О внесении изменений в Федеральный закон \\"О федеральном бюджете на 2020 год и на плановый период 2021 и 2022 годов\\"","comments":"об уточнении параметров федерального бюджета","introductionDate":"2020-02-19","url":"http:\\/\\/sozd.parlament.gov.ru\\/bill\\/904447-7","transcriptUrl":"http:\\/\\/api.duma.gov.ru\\/api\\/transcript\\/904447-7","lastEvent":{"stage":{"id":8,"name":"Прохождение закона у Президента Российской Федерации"},"phase":{"id":23,"name":"Рассмотрение закона Президентом Российской Федерации"},"solution":"закон подписан","date":"2020-03-18","document":null},"subject":{"deputies":[],"departments":[{"id":6230800,"name":"Правительство РФ","isCurrent":true,"startDate":"1994-01-01","endDate":null}],"factions":[]},"committees":{"responsible":{"id":6276700,"name":"Комитет ГД по бюджету и налога